## Import

In [1]:
import cv2
from PIL import Image
import numpy as np
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from tensorflow.keras.models import  load_model
import joblib

# mapping_label = {'10c':0,
#   '20c':1,
#   '5c':2}`sw

## CNN

In [2]:
MODEL_NAME = 'cnn-combined_updated.hd5'
model = load_model(MODEL_NAME)

img = Image.open('./test_dataset/test-image3.jpg')
img=img.rotate(4)
area = (400, 250, 950, 800)
img = img.crop(area)

pil_image = img.convert('RGB') 
open_cv_image = np.array(pil_image) 
# Convert RGB to BGR 
resized_image = open_cv_image[:, :, ::-1].copy() 

# To pass these images into a neural network, the images will need to be the same size
resized_image = cv2.resize(resized_image, (300, 300))
resized_image = np.expand_dims(resized_image, axis=0)

y_prob = model.predict(resized_image) 
y_prob_dec = [format(float(prob), '.8f') for prob in y_prob[0]]
print('Probability:',y_prob_dec)
y_classes = y_prob.argmax(axis=-1)
print('Predicted class:', y_classes, '\n')

cnn_pred = y_classes[0]

1/1 [==============================] - 0s 127ms/step
Probability: ['0.00008307', '0.99981219', '0.00010482']
Predicted class: [1] 



## RF

In [3]:
rf = joblib.load('rf.pkl')

img = Image.open('./test_dataset/test-image3.jpg')
img=img.rotate(4)
area = (400, 250, 950, 800)
img = img.crop(area)

pil_image = img.convert('RGB') 
open_cv_image = np.array(pil_image) 
# Convert RGB to BGR 
resized_image = open_cv_image[:, :, ::-1].copy() 
resized_image = cv2.resize(resized_image, (300, 300))
resized_image = np.expand_dims(resized_image, axis=0)

nsamples, nx, ny, nrgb = resized_image.shape
x_test = resized_image.reshape((nsamples,nx*ny*nrgb))
y_prob = rf.predict(x_test) 
y_prob_dec = [format(float(prob), '.8f') for prob in y_prob[0]]
print('Probability:',y_prob_dec)
y_classes = y_prob.argmax(axis=-1)
print('Predicted class:', y_classes, '\n')

rf_pred = y_classes[0]

Probability: ['1.00000000', '0.00000000', '0.00000000']
Predicted class: [0] 



## KNN + HoughCircles

In [4]:
knn = joblib.load('knn_hough.pkl')

img = Image.open('./test_dataset/test-image3.jpg')
img=img.rotate(4)
area = (400, 250, 950, 800)
img = img.crop(area)

pil_image = img.convert('RGB') 
open_cv_image = np.array(pil_image) 
# Convert RGB to BGR 
resized_image = open_cv_image[:, :, ::-1].copy() 

# Convert to grayscale.
gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

# Blur using 3 * 3 kernel.
gray_blurred = cv2.blur(gray, (3, 3))

# Apply Hough transform on the blurred image.
detected_circles = cv2.HoughCircles(gray_blurred, 
                   cv2.HOUGH_GRADIENT, 1, 50, param1 = 100,
               param2 = 100, minRadius = 100, maxRadius = 300)

# Draw circles that are detected.
if detected_circles is not None:

    # Convert the circle parameters a, b and r to integers.
    detected_circles = np.uint16(np.around(detected_circles))

    for pt in detected_circles[0]:
        a, b, r = pt[0], pt[1], pt[2]
        break
print(type(r))
print(type(np.uint16(5)))
x_test = r.reshape(-1,1)
print(x_test)
y_prob = knn.predict(x_test) 
y_prob_dec = [format(float(prob), '.8f') for prob in y_prob[0]]
print('Probability:',y_prob_dec)
y_classes = y_prob.argmax(axis=-1)
print('Predicted class:', y_classes, '\n')

knn_pred = y_classes[0]

<class 'numpy.uint16'>
<class 'numpy.uint16'>
[[196]]
Probability: ['0.00000000', '1.00000000', '0.00000000']
Predicted class: [1] 



In [5]:
# Function to decide which coin is predicted ultimately by combining the 3 models. KNN is given the most priority as it is the most accurate.
def weighted_voting(cnn_pred,knn_pred,rf_pred):
    cnn_weight = 0.25
    knn_weight = 0.5
    rf_weight = 0.25
    model = {}
    try:
        model[cnn_pred] += cnn_weight
    except:
        model[cnn_pred] = cnn_weight
    try:
        model[knn_pred] += knn_weight
    except:
        model[knn_pred] = knn_weight
    try:
        model[rf_pred] += rf_weight
    except:
        model[rf_pred] = rf_weight
        
    max_val = max(model, key=model.get)
    min_val = min(model, key=model.get)
    print(model)
    if max_val == min_val and len(model) != 1: #if there is a tiebreak, use knn results since rf can be very wrong
        return knn_pred
    else:
        return max_val
print(cnn_pred,knn_pred,rf_pred)
weighted_voting(cnn_pred,knn_pred,rf_pred)

1 1 0
{1: 0.75, 0: 0.25}


1